In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
quote_text = pd.read_csv('GNI88.csv')

In [3]:
article_data = pd.read_json("gni88.json", lines=True)
article_data.head()

,Article ID,Headline,Content,Media Name,Author,Published Date
0,3777409,Modernized Tu-160 to boost Russia's long-range...,Modernized Tu-160 to boost Russia's long-range...,Defense News,"Bodner, Matthew",2018-02-15 00:00:00+00
1,3764250,Give Trump more nuclear weapons and more ways ...,Give Trump more nuclear weapons and more ways ...,CNN,"Collina, Tom Z.",2018-02-02 00:00:00+00
2,3764251,Trump wants to build smaller atomic weapons; R...,Trump wants to build smaller atomic weapons; R...,Los Angeles Times,"Cloud, David",2018-02-03 00:00:00+00
3,3764252,Egypt and Israel Secretly Allied In Sinai Battle,Egypt and Israel Secretly Allied In Sinai Batt...,The New York Times,"Kirkpatrick, David D.",2018-02-04 00:00:00+00
4,3764253,Listen: Top Armed Services Dem worried about r...,Listen: Top Armed Services Dem worried about r...,The Hill,"Simendinger, Alexis",2018-02-03 00:00:00+00


In [4]:
len(article_data)

69995

In [ ]:
# filter the article_data


# Extract first 10% of article content

make new content column -- does not include title, author, date

In [5]:
# only take everything after date in content
def justContent(content):
    content = ''.join(content.split('\r\n\r\n')[2:])
    return content

In [6]:
article_data['Content2'] = article_data['Content'].apply(lambda x: justContent(x))

In [7]:
article_data['First_10%_Content_Split'] = article_data['Content2'].apply(lambda x: x.split()[:int(0.1*len(x))])

In [8]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
article_data['First_10%_Content'] = article_data['First_10%_Content_Split'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

In [9]:
# takes only columns with content
df = article_data.loc[article_data['First_10%_Content_Split'].apply(lambda x: len(x) >0)]

In [10]:
merged_df = quote_text.merge(df[['Article ID', 'Content', 'Author', 'Published Date', 'First_10%_Content', 'First_10%_Content_Split']], on = 'Article ID')

- percentage of words overlapped (1st 10%, Messages) ~ 0.53
- percentage of words overlapped (1st 10%, Submessages) ~ 0.41

In [28]:
def num_overlapped_words(col1, col2):
    return [len(set(x).intersection(y)) for x, y in zip(merged_df[col1], merged_df[col2])]
def prop_overlapped_words(col1, col2):
    len_article_issues = merged_df[col1].apply(lambda x: len(x))
    #merged_df['Article Issues Split'].apply(lambda x: len(x))
    return num_overlapped_words(col1, col2) / len_article_issues

In [15]:
merged_df['Article Issues Split'] =  merged_df['Article Issues'].apply(lambda x: str(x).split())

In [16]:
merged_df['Messages Split'] =  merged_df['Messages'].apply(lambda x: str(x).split())

In [18]:
merged_df['First_10%_Content Split'] =  merged_df['First_10%_Content'].apply(lambda x: str(x).split())

In [29]:
merged_df['Prop Overlapped (Messages, 10% Content)'] = prop_overlapped_words('Messages Split', 'First_10%_Content Split')


In [30]:
messages_first10 = np.mean(merged_df['Prop Overlapped (Messages, 10% Content)'])
messages_first10

0.2613437535612828

In [31]:
merged_df['Prop Overlapped (Messages, 10% Content)']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
301268    0.0
301269    0.0
301270    0.0
301271    0.0
301272    0.0
Name: Prop Overlapped (Messages, 10% Content), Length: 301273, dtype: float64

In [24]:
merged_df['Submessages Split'] =  merged_df['Submessages'].apply(lambda x: str(x).split())

In [32]:
merged_df['Prop Overlapped (Submessages, 10% Content)'] = prop_overlapped_words('Submessages Split', 'First_10%_Content Split')


In [33]:
submessages_first10 = np.mean(merged_df['Prop Overlapped (Submessages, 10% Content)'])
submessages_first10

0.08310120721076701

In [27]:
merged_df['Article Issues Split']

0                     [nan]
1                     [nan]
2                     [nan]
3                     [nan]
4                     [nan]
                ...        
301268    [Arms, Reduction]
301269    [Arms, Reduction]
301270                [nan]
301271                [nan]
301272                [nan]
Name: Article Issues Split, Length: 301273, dtype: object

In [34]:
# only include the feature 'Prop Overlapped (Messages, 10% Content)'

In [36]:
#merged_df['Prop Overlapped (Messages, 10% Content)']

In [38]:
merged_df['Headline Split'] = merged_df['Headline'].apply(lambda x: str(x).split())

In [39]:
merged_df['Prop Overlapped (Messages, Headline)'] = prop_overlapped_words('Messages Split', 'Headline Split')


In [40]:
messages_headline = np.mean(merged_df['Prop Overlapped (Messages, Headline)'])
messages_headline

0.15377553470329786

In [87]:
proportion_features = merged_df[['Article ID', 'Prop Overlapped (Messages, 10% Content)', 'Prop Overlapped (Messages, Headline)']]

# Treaties Features

In [43]:
treaty_message_table = pd.read_csv('treaty_message_table')

In [44]:
treaty_message_table

,Unnamed: 0,treaty,most common message
0,0,the nuclear nonproliferation treaty,OLD Iran
1,1,the nonproliferation treaty,OLD Iran
2,2,the north atlantic treaty,OLD Missile Defense
3,3,the new start treaty,OLD Arms Reduction
4,4,the intermediaterange nuclear forces treaty,OLD Arms Reduction
...,...,...,...
127,127,the arms trade treaty,OLD Missile Defense
128,128,the panama canal treaty,OLD Iran
129,129,the iran nuclear treaty,OLD Proliferation
130,130,the oslo treaty,OLD Proliferation


In [49]:
treaty_articles = pd.read_csv('merged_df')

In [54]:
treaty_articles = treaty_articles.loc[treaty_articles['Treaties2'].isin(treaty_message_table['treaty'])]

message

In [55]:
predicted_message = []
for i in np.arange(len(treaty_articles)):
    #if treaty_articles.reset_index()['Treaties2'][i] == 
    # find treaty in treaty_message_table, append most common message
    treaty = treaty_articles.reset_index()['Treaties2'][i]
    treaty_message = treaty_message_table.loc[treaty_message_table['treaty'] == treaty]['most common message'].item()
    predicted_message.append(treaty_message)
    

In [56]:
true_predicted_message_table = treaty_articles[['Messages']].reset_index(drop=True)
true_predicted_message_table['predicted_message'] = predicted_message
true_predicted_message_table = true_predicted_message_table.rename(columns= {'Messages': 'true_message'})

In [57]:
true_predicted_message_table

,true_message,predicted_message
0,OLD Arms Reduction,OLD Arms Reduction
1,OLD Arms Reduction,OLD Arms Reduction
2,OLD Arms Reduction,OLD Arms Reduction
3,OLD Proliferation,OLD Iran
4,OLD Proliferation,OLD Iran
...,...,...
10584,OLD Arms Reduction,OLD Arms Reduction
10585,OLD Proliferation,OLD Arms Reduction
10586,OLD Missile Defense,OLD Missile Defense
10587,OLD Arms Reduction,OLD Missile Defense


In [58]:
correct = sum(true_predicted_message_table['true_message'] == true_predicted_message_table['predicted_message'])
total = len(true_predicted_message_table)
accuracy = correct/total
accuracy

0.5761639437151761

In [61]:
treaty_articles['predicted_message'] = true_predicted_message_table['predicted_message']

In [63]:
treaty_articles['predicted_message'].unique()

array(['OLD Arms Reduction', 'OLD Iran', 'OLD Proliferation',
       'OLD Missile Defense', 'OLD Foreign Policy Narrative', nan],
      dtype=object)

In [62]:
treaty_articles[['Article ID', 'predicted_message']]

,Article ID,predicted_message
83,3784488,OLD Arms Reduction
84,3784488,OLD Arms Reduction
85,3784488,OLD Arms Reduction
461,3897492,OLD Arms Reduction
462,3897492,OLD Arms Reduction
...,...,...
301256,6389880,NaN
301257,6389880,NaN
301263,7303114,NaN
301264,7303114,NaN


submessage

In [67]:
submessages_treaties = treaty_articles[['Treaties2', 'Submessages']]

In [69]:
def submessage_treaty_table(treaty, topic_column):
    table =  treaty_articles.loc[treaty_articles['Treaties2'] == treaty].groupby(by = topic_column).count()
    return table.reset_index().iloc[:,0:2].rename(columns = {'Unnamed: 0' : 'count'})

In [70]:
submessage_treaty_table('the nuclear nonproliferation treaty', 'Submessages')

,Submessages,count
0,NPT,1
1,OLD Politics,11
2,OLD AQ Khan Network,2
3,OLD Arms Reduction,117
4,OLD CTBT,5
5,OLD Covert Ops,16
6,OLD Deployment and Testing,48
7,OLD Domestic Programs,6
8,OLD European/Russian Missile Defense,22
9,OLD Fissile Material Cut Off Treaty,3


In [71]:
treaties_unique = pd.DataFrame(treaty_articles['Treaties2'].value_counts())
treaties_unique#.loc[treaties_unique['Treaties2'] > 50]

,Treaties2
the nuclear nonproliferation treaty,4043
the nonproliferation treaty,879
the north atlantic treaty,741
the new start treaty,545
the intermediaterange nuclear forces treaty,543
...,...
the oslo treaty,1
the arms trade treaty,1
the panama canal treaty,1
"the summit, north atlantic treaty",1


In [72]:
def submessage_treaty_props(treaty, topic_column):
    table =  treaty_articles.loc[treaty_articles['Treaties2'] == treaty].groupby(by = topic_column).count()
    table = table.reset_index().iloc[:,0:2].rename(columns = {'Unnamed: 0' : 'count'})
    total = sum(table['count'])
    props = []
    for i in np.arange(len(table)):
        props.append(table['count'][i] / total)
    table['props'] = props
    table['treaty'] = treaty
    return table[['treaty', topic_column, 'props']]


In [73]:
treaties_list = treaties_unique.index

In [74]:
treaty_submessage_table = submessage_treaty_props(treaties_list[0], 'Submessages')
treaty_submessage_table= pd.pivot_table(treaty_submessage_table, values = 'props', index = ['treaty'], columns = ['Submessages'])

for i in treaties_list[1:]:
    table = submessage_treaty_props(i, 'Submessages')
    table = pd.pivot_table(table, values = 'props', index = ['treaty'], columns = ['Submessages'])
    treaty_submessage_table = pd.concat([treaty_submessage_table, table])

In [75]:
pd.set_option('display.max_rows', len(treaty_submessage_table))
treaty_submessage_table

Submessages,NPT,OLD Politics,OLD AQ Khan Network,OLD Arms Reduction,OLD CTBT,OLD Covert Ops,OLD Deployment and Testing,OLD Domestic Programs,OLD European/Russian Missile Defense,OLD Fissile Material Cut Off Treaty,...,OLD Sanctions,OLD State Level Nuclear Programs,OLD Tactical Nukes Treaty,OLD Terrorist Threat,OLD US Arsenal,US Nuclear Arsenal,OLD INF Treaty,OLD Spending,OLD NSA Budget,OLD Sequestration
treaty,,,,,,,,,,,,,,,,,,,,,
the nuclear nonproliferation treaty,0.000247,0.002721,0.000495,0.028939,0.001237,0.003957,0.011872,0.001484,0.005442,0.000742,...,0.054662,0.245115,0.003463,0.003710,0.009646,0.000247,NaN,NaN,NaN,NaN
the nonproliferation treaty,NaN,0.003417,NaN,0.053531,NaN,0.003417,NaN,0.003417,0.001139,NaN,...,0.048975,0.216401,NaN,NaN,0.005695,NaN,NaN,NaN,NaN,NaN
the north atlantic treaty,NaN,0.096467,NaN,0.004076,NaN,0.002717,0.058424,0.008152,0.326087,NaN,...,0.027174,0.142663,0.024457,0.001359,0.032609,NaN,0.001359,0.017663,NaN,NaN
the new start treaty,NaN,0.031193,NaN,0.066055,0.005505,NaN,0.007339,0.016514,0.139450,NaN,...,0.003670,0.033028,0.003670,0.005505,0.102752,NaN,NaN,NaN,0.023853,0.007339
the intermediaterange nuclear forces treaty,NaN,0.037244,NaN,0.178771,NaN,NaN,0.022346,0.050279,0.122905,NaN,...,0.003724,0.117318,NaN,NaN,0.054004,NaN,0.040968,NaN,NaN,NaN
the treaty,NaN,0.017045,NaN,0.034091,0.071023,NaN,0.028409,0.002841,0.025568,0.014205,...,0.028409,0.269886,0.053977,NaN,0.045455,NaN,NaN,NaN,NaN,NaN
the comprehensive test ban treaty,NaN,0.002959,NaN,0.065089,0.556213,NaN,0.026627,0.005917,0.014793,NaN,...,0.002959,0.147929,NaN,0.005917,0.011834,NaN,NaN,NaN,0.002959,NaN
the antiballistic missile treaty,NaN,0.043137,NaN,0.086275,NaN,NaN,0.031373,0.168627,0.282353,NaN,...,NaN,0.035294,NaN,NaN,0.035294,NaN,NaN,NaN,NaN,NaN
the inf treaty,NaN,0.004878,NaN,0.160976,NaN,NaN,0.004878,0.087805,0.180488,NaN,...,0.004878,0.063415,0.004878,0.004878,0.068293,NaN,0.204878,NaN,NaN,NaN


In [76]:
# table with treaty & most common message (highest prop)
most_common_submessage = []
for i in np.arange(len(treaty_submessage_table)):
    most_common_submessage.append(pd.DataFrame(treaty_submessage_table.iloc[i, :]).idxmax()[0])

In [77]:
treaty_submessage_table['most common submessage'] = most_common_submessage

In [78]:
treaty_submessage_table = treaty_submessage_table.reset_index()[['treaty', 'most common submessage']]
treaty_submessage_table = treaty_submessage_table.rename_axis(None, axis=1)

In [79]:
treaty_submessage_table

,treaty,most common submessage
0,the nuclear nonproliferation treaty,OLD Negotiations
1,the nonproliferation treaty,OLD Negotiations
2,the north atlantic treaty,OLD European/Russian Missile Defense
3,the new start treaty,OLD START Treaty
4,the intermediaterange nuclear forces treaty,OLD START Treaty
5,the treaty,OLD State Level Nuclear Programs
6,the comprehensive test ban treaty,OLD CTBT
7,the antiballistic missile treaty,OLD European/Russian Missile Defense
8,the inf treaty,OLD INF Treaty
9,the comprehensive nucleartestban treaty,OLD State Level Nuclear Programs


In [80]:
predicted_submessage = []
for i in np.arange(len(treaty_articles)):
    # find treaty in treaty_message_table, append most common message
    treaty = treaty_articles.reset_index()['Treaties2'][i]
    treaty_submessage = treaty_submessage_table.loc[treaty_submessage_table['treaty'] == treaty]['most common submessage'].item()
    predicted_submessage.append(treaty_submessage)
    

In [81]:
true_predicted_submessage_table = treaty_articles[['Submessages']].reset_index(drop=True)
true_predicted_submessage_table['predicted_submessage'] = predicted_submessage
true_predicted_submessage_table = true_predicted_submessage_table.rename(columns= {'Submessages': 'true_submessage'})

In [82]:
true_predicted_submessage_table

,true_submessage,predicted_submessage
0,OLD Arms Reduction,OLD Arms Reduction
1,OLD Arms Reduction,OLD Arms Reduction
2,OLD Arms Reduction,OLD Arms Reduction
3,OLD State Level Nuclear Programs,OLD Negotiations
4,OLD State Level Nuclear Programs,OLD Negotiations
...,...,...
10584,OLD START Treaty,OLD INF Treaty
10585,OLD State Level Nuclear Programs,OLD INF Treaty
10586,OLD Deployment and Testing,OLD European/Russian Missile Defense
10587,OLD INF Treaty,OLD European/Russian Missile Defense


In [83]:
treaty_articles['predicted_submessage'] = true_predicted_submessage_table['predicted_submessage']

In [89]:
treaty_features = treaty_articles[['Article ID', 'predicted_message', 'predicted_submessage']]

In [90]:
treaty_features

,Article ID,predicted_message,predicted_submessage
83,3784488,OLD Arms Reduction,OLD CTBT
84,3784488,OLD Arms Reduction,OLD CTBT
85,3784488,OLD Arms Reduction,OLD CTBT
461,3897492,OLD Arms Reduction,OLD START Treaty
462,3897492,OLD Arms Reduction,OLD START Treaty
...,...,...,...
301256,6389880,NaN,NaN
301257,6389880,NaN,NaN
301263,7303114,NaN,NaN
301264,7303114,NaN,NaN


In [91]:
proportion_features

,Article ID,"Prop Overlapped (Messages, 10% Content)","Prop Overlapped (Messages, Headline)"
0,3759306,0.0,0.0
1,3759306,0.0,0.0
2,3759306,0.0,0.0
3,3759306,0.0,0.0
4,3759306,0.0,0.0
...,...,...,...
301268,7388318,0.0,0.0
301269,7388318,0.0,0.0
301270,7532855,0.0,0.0
301271,7532855,0.0,0.0


In [98]:
all_features_table = proportion_features.merge(treaty_features, on= 'Article ID')

In [131]:
all_features_table

,Article ID,"Prop Overlapped (Messages, 10% Content)","Prop Overlapped (Messages, Headline)",predicted_message,predicted_submessage
0,3784488,0.000000,0.0,OLD Arms Reduction,OLD CTBT
1,3784488,0.000000,0.0,OLD Arms Reduction,OLD CTBT
2,3784488,0.000000,0.0,OLD Arms Reduction,OLD CTBT
3,3784488,0.000000,0.0,OLD Arms Reduction,OLD CTBT
4,3784488,0.000000,0.0,OLD Arms Reduction,OLD CTBT
...,...,...,...,...,...
150924,7303114,0.000000,0.0,NaN,NaN
150925,7303114,0.000000,0.0,NaN,NaN
150926,7303114,0.333333,0.0,NaN,NaN
150927,7303114,0.333333,0.0,NaN,NaN


In [132]:
final_features_table = all_features_table.groupby(by = 'Article ID').first()

In [133]:
final_features_table

,"Prop Overlapped (Messages, 10% Content)","Prop Overlapped (Messages, Headline)",predicted_message,predicted_submessage
Article ID,,,,
3761315,0.666667,0.0,None,None
3761316,0.666667,0.0,None,None
3762531,0.000000,0.0,None,None
3763395,0.000000,0.0,None,None
3764265,0.000000,0.0,None,None
...,...,...,...,...
7426939,0.000000,0.0,None,None
7439405,0.000000,1.0,None,None
7469879,0.000000,0.0,None,None
